---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
# Incorrect according to auto-grader, but didn't seem to impact the actual ttest..
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''        
    
    df = pd.read_table('university_towns.txt', header=None, sep='\n', names = ['name'])
    
    # State names in university_towns.txt all end with [edit]
    # lambda expression assigns State value NaN to all regions, and ffill() helps propagates the state value all
    # regions in a state by filling missing values with last valid value.
    df['State'] = df['name'].apply(lambda x: x[:-6].strip()
                                   if x.endswith("[edit]") else np.nan).ffill()
    
    # All university town name entries end with a parenthesis containing names of universities located there.
    df['RegionName'] = df['name'].apply(lambda x: x.split("(", maxsplit=1)[0].strip()
                                        if x.count("(") > 0 else np.nan)
    
    return df[['State', 'RegionName']].dropna(axis='index')

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # Data for 2000Q1 is located on row 221. First four columns are annual data and whitespace, thus can be skipped.
    # Two GDP columns represent GDP in billion USD current dollars, and 2009 chained dollars, respectively.
    # Using the inflation-adjusted column for purpose of this assignment.
    df = pd.read_excel('gdplev.xls', skiprows=220, usecols=[4, 6],
                       header=None, names=['Quarter', 'GDP'])
    
    # shift() moves all rows down by 1 (default period=1), essentially comparing each quarter's GDP with that of
    # the previous quarter. Similarly, shift(-1) moves all rows up by 1, and compares with the next quarter.
    matched = df[(df['GDP'] < df['GDP'].shift())&
                (df['GDP'] > df['GDP'].shift(-1))]
    return matched.iloc[0, 0]

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=220, usecols=[4, 6],
                       header=None, names=['Quarter', 'GDP'])
    
    start = get_recession_start()
    
    matched = df[(df['GDP'] > df['GDP'].shift()) &
                (df['GDP'].shift() > df['GDP'].shift(2)) &
                (df['Quarter'] > start)]
       
    return matched.iloc[0, 0]

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', skiprows=220, usecols=[4, 6],
                       header=None, names=['Quarter', 'GDP'])

    start_idx = df.loc[df['Quarter'] == get_recession_start()].index.values.astype(int)[0]
    end_idx = df.loc[df['Quarter'] == get_recession_end()].index.values.astype(int)[0] + 1
    bottom = ''
    min_gdp = np.finfo(np.float128).max
    
    for i in range(start_idx, end_idx):
        if df.iloc[i]['GDP'] < min_gdp:
            min_gdp = df.iloc[i]['GDP']
            bottom =  df.iloc[i]['Quarter']
    
    return bottom

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    # Housing data covers 1996-04 to 2016-08.
    # Don't need columns 0, 3, 4, 5 (RegionID, Metro, CountyName, SizeRank)
    cols = list(pd.read_csv('City_Zhvi_AllHomes.csv', nrows=1))
    cols = [x for x in cols
            if x in ['RegionName', 'State'] or x.startswith('20')]
    prices = pd.read_csv('City_Zhvi_AllHomes.csv', usecols=cols)
    
    prices = prices.replace({'State': states})
    prices = prices.set_index(['State', 'RegionName'])
    
    # https://stackoverflow.com/a/40497246
    prices = (prices.groupby(pd.PeriodIndex(prices.columns, freq='Q'), axis=1)
              .mean()
              .rename(columns=lambda c: str(c).lower()))
    
    return prices

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    prices = convert_housing_data_to_quarters().reset_index()
    start = get_recession_start()
    bottom = get_recession_bottom()
    towns = get_list_of_university_towns()
    
    # when indicator is set to true, the merged data frame will include an extra column "_merge"
    # indicating source of the row. Valid values are "left_only", "right_only", and "both"
    df = pd.merge(prices, towns, on=['State','RegionName'], how='left', indicator=True)
    df['Ratio'] = df[start] / df[bottom]
    u = df[df['_merge'] == 'both']
    nu = df[df['_merge'] == 'left_only']
    
    from scipy import stats
    t, p = stats.ttest_ind(nu['Ratio'].dropna(), u['Ratio'].dropna())
    different = (p < 0.01)
    better = "university town" if (u['Ratio'].mean() < nu['Ratio'].mean()) else "non-university town"
    
    return different, p, better